In [1]:
from selenium                           import webdriver
from webdriver_manager.chrome           import ChromeDriverManager
from selenium.webdriver.chrome.service  import Service
from bs4                                import BeautifulSoup
from selenium.webdriver.chrome.options  import Options
from selenium.webdriver.common.by       import By
from time                               import sleep
import pandas                           as pd


#Definição da Cidade-------------------------------------

cidade = 'Campo Magro'

with open(f'bairros{cidade}.txt','r', encoding = 'utf-8') as arquivo:
    bairros = arquivo.readlines()

print(bairros)


#Abrindo navegador---------------------------------------------------------------------------------------

servico = Service(ChromeDriverManager().install())

options = Options()
options.add_argument('--headless')
options.add_argument("window-size= 800,800")
options.add_argument('service=servico')

navegador = webdriver.Chrome(options=options)



def pesquisar(bairro, cidade):
    navegador.get("https://www.google.com.br/maps/")
    pesquisa = navegador.find_element(By.XPATH, '//*[@id="searchboxinput"]')
    pesquisa.send_keys(f'escolas em {cidade} {bairro}')
    pesquisa.submit()
    
    sleep(10)

#MONTANDO AS LISTAS------------------------------------------------------------------------------------------
    
lista_nomeleads=[]
lista_sitemaps =[]
lista_nTefone  =[]
lista_nWhats   =[]
lista_endereco =[]
lista_site     =[]
lista_ativos   =[]
lista_bairro   =[]
lista_pesquisa =[]    
    
    
for bairro in bairros:
    
    pesquisar(bairro,cidade)
    
    page_content = navegador.page_source

    site = BeautifulSoup(page_content, 'html.parser')
    a = 0
    
    
    lista_pesquisa = []
    #ENCONTRANDO LEADS COM MAPS---------------------------------------------------------------------------------
    
    
    anuncios = site.findAll('div', attrs={'class': 'Nv2PK tH5CWc THOPZb'})
    
    for anuncio in anuncios:
        nome_comercio = anuncio.find('a', attrs={'class': 'hfpxzc'})['aria-label']
        site_ref = anuncio.find('a', attrs={'class': 'hfpxzc'})['href']
        a = a + 1
        
        lista_pesquisa.append(site_ref)
        #print(nome_comercio)
        #print(site_ref)
#        print()
    print(a)
    
    for enderecos in lista_pesquisa:
        navegador.get(f'{enderecos}')
        
        page_content = navegador.page_source
    
        site = BeautifulSoup(page_content, 'html.parser')
        sleep(5)
        
        nome_comerciall = site.find('h1', attrs={'class':'DUwDvf fontHeadlineLarge'})
    
        if nome_comerciall != None:
            nome_comercial = nome_comerciall.find('span').text
            print('nome comercial:',nome_comercial)
        else:
            nome_comercial = None
        
        dados = site.findAll('div', attrs={'class': 'RcCsl fVHpi w4vB1d NOE9ve M0S7ae AG25L'})
    
        
    
        local  = site.find('button', attrs={'data-tooltip': 'Copiar endereço'})
        
        if local != None:
            endereco_comercial = local.find('div', attrs={'class': 'Io6YTe fontBodyMedium'}).text
#            print('end: ',endereco_comercial)
        else:
            endereco_comercial = None
    

        whats    = site.find('a', attrs={'aria-label': 'whatsapp.com'})

        if whats != None:
            whats_comercio = whats['href']
#            print('whats: ',whats_comercio)   
        else:
            whats_comercio = None

    

        sitee    = site.find('a', attrs={'data-tooltip': 'Abrir website'})


        if sitee != None:
            site_comercio = sitee['href']
#            print('site:',site_comercio)
        else:
            site_comercio = None





        telefono = site.find('button', attrs={'data-tooltip': 'Copiar número de telefone'})

        if telefono != None:
            telefone_comercio  = telefono.find('div', attrs={'class':'Io6YTe fontBodyMedium'}).text

#            print('tel:',telefone_comercio)
        else:
            telefone_comercio = None


        ativos = 0

#        print('maps: ',enderecos)   

    
    
        lista_nomeleads.append(nome_comercial)
        lista_nTefone.append(telefone_comercio)
        lista_nWhats.append(whats_comercio)
        lista_site.append(site_comercio)
        lista_endereco.append(endereco_comercial)
        lista_sitemaps.append(enderecos)
        lista_ativos.append(ativos)
        lista_bairro.append(bairro)






        endereco_comercial = None
        nome_comercial     = None
        whats_comercio     = None
        site_comercio      = None
        telefone_comercio  = None
        endereco_maps      = None
        ativos             = None
        #print()
        #print()    
        #print()
#Montando Data Frame-----------------------------------------------------------------------------------------
df_leads = pd.DataFrame(data={'Leads(Parceiros)':lista_nomeleads,
                                   'Ativos (0/1)':lista_ativos,
                                   'Telefone':lista_nTefone,
                                   'Link Whatsapp':lista_nWhats,
                                   'Link Site':lista_site,
                                   'Endereço':lista_endereco,
                                   'Link Maps':lista_sitemaps,
                                    'Bairro': lista_bairro})

df_leads_ok = df_leads.drop_duplicates(subset= 'Leads(Parceiros)')


#Criando Arquivo Excel com os Leads---------------------------------------------------------------------------

df_leads_ok.to_excel(f'Leads_escolas{cidade}.xlsx', index = False)




['Água Boa\n', 'Boa Vista\n', 'Centro\n', 'Jardim Bom Pastor\n', 'Jardim Novos Horizontes\n', 'Jardim Viviane']
3
nome comercial: Colégio Estadual Campo Magro
nome comercial: Colégio Estadual Emilia Buzato
nome comercial: Campo Magro, C E-Ef M
2
nome comercial: Escola Municipal Sagrada Família
nome comercial: Colégio Estadual Campo Magro
2
nome comercial: Escola Municipal Sagrada Família
nome comercial: Colégio Estadual Campo Magro
0
2
nome comercial: Escola Municipal Sagrada Família
nome comercial: Colégio Estadual Campo Magro
0


In [2]:
df_leads


,Leads(Parceiros),Ativos (0/1),Telefone,Link Whatsapp,Link Site,Endereço,Link Maps,Bairro
0,Colégio Estadual Campo Magro,0,(41) 3677-5565,None,http://www.campomagro.pr.gov.br/educacao/,"Av. Rancho Alegre, 150 - Jardim Pioneiro, Camp...",https://www.google.com.br/maps/place/Col%C3%A9...,Água Boa\n
1,Colégio Estadual Emilia Buzato,0,(41) 3677-7710,None,http://www.camgemilia.seed.pr.gov.br/,"Estrada do Cerne, km13,5, Estr. do Cerne, Camp...",https://www.google.com.br/maps/place/Col%C3%A9...,Água Boa\n
2,"Campo Magro, C E-Ef M",0,(41) 3677-5565,None,http://www.consultaescolas.pr.gov.br/consultae...,"Av. Rancho Alegre, 150 - Jardim Pioneiro, Camp...",https://www.google.com.br/maps/place/Campo+Mag...,Água Boa\n
3,Escola Municipal Sagrada Família,0,(41) 3677-1385,None,http://www.campomagro.pr.gov.br/educacao/,"Estrada do Cerne, Km 19,5, s/n - Centro, Campo...",https://www.google.com.br/maps/place/Escola+Mu...,Boa Vista\n
4,Colégio Estadual Campo Magro,0,(41) 3677-5565,None,http://www.campomagro.pr.gov.br/educacao/,"Av. Rancho Alegre, 150 - Jardim Pioneiro, Camp...",https://www.google.com.br/maps/place/Col%C3%A9...,Boa Vista\n
5,Escola Municipal Sagrada Família,0,(41) 3677-1385,None,http://www.campomagro.pr.gov.br/educacao/,"Estrada do Cerne, Km 19,5, s/n - Centro, Campo...",https://www.google.com.br/maps/place/Escola+Mu...,Centro\n
6,Colégio Estadual Campo Magro,0,(41) 3677-5565,None,http://www.campomagro.pr.gov.br/educacao/,"Av. Rancho Alegre, 150 - Jardim Pioneiro, Camp...",https://www.google.com.br/maps/place/Col%C3%A9...,Centro\n
7,Escola Municipal Sagrada Família,0,(41) 3677-1385,None,http://www.campomagro.pr.gov.br/educacao/,"Estrada do Cerne, Km 19,5, s/n - Centro, Campo...",https://www.google.com.br/maps/place/Escola+Mu...,Jardim Novos Horizontes\n
8,Colégio Estadual Campo Magro,0,(41) 3677-5565,None,http://www.campomagro.pr.gov.br/educacao/,"Av. Rancho Alegre, 150 - Jardim Pioneiro, Camp...",https://www.google.com.br/maps/place/Col%C3%A9...,Jardim Novos Horizontes\n


In [ ]:
navegador.close()